In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data 
rest_df = pd.read_csv('data/test/restaurants.csv')
locs_df = pd.read_csv('data/test/locations.csv')

In [15]:
# Display the restaurant data
rest_df

,Name,Maps Link,Closed,Max Delivery Distance (miles),Max Deliveries/week,Max Deliveries/day
0,Cafe Ynez,https://g.page/cafeynez?share,M,5,4,2
1,Rex 1516,https://goo.gl/maps/rUymnTPQYhuNyH5V6,"M, T",10,3,2
2,El Merkury,https://goo.gl/maps/tejVM4WbUxguxpfe9,"S, Su",2,3,2
3,On Point,https://goo.gl/maps/CNyUQfXe59PdPmVv6,F,5,4,2
4,Baology,https://g.page/baology?share,None,5,5,1
5,Sidecar,https://goo.gl/maps/cnKkYkWhnVcgRSAK8,None,5,5,1
6,Double Knot,https://goo.gl/maps/ptfobLMFXftCsZ9w5,None,10,5,2
7,Destination Dogs,https://goo.gl/maps/zf6RYMmB9rJnrK2E9,Th,10,4,3
8,Erawan,https://goo.gl/maps/8f4YEWcWUazPLLXLA,M,5,3,2
9,Sabrina's Cafe,https://goo.gl/maps/TEY74TYC7Ja1wvX27,W,2,3,1


In [17]:
# Display the location data
locs_df

,Name,Maps Link
0,HUP,https://goo.gl/maps/UNaP3MudA9wwRYFB6
1,Presby,https://goo.gl/maps/T4yAjq7itGHbCvXN7
2,CHOP,https://goo.gl/maps/HE7PAY6fPfNRhe1LA
3,Jefferson,https://goo.gl/maps/rDBFBTSbyiqftmvG7


In [3]:
# Filter out the restaurants based on day of the week, number of deliveries per week
DAY = 'M'
def filter_day(row):
    closed_days = row['Closed'].replace(' ', '').split(',')
    if not (DAY in closed_days):
        return True
    else:
        return False
rest_df_f = rest_df[rest_df.apply(filter_day, axis=1)]

# TO DO: Filter based on number of deliveries already completed for the week using db

In [18]:
rest_df_f

,Name,Maps Link,Closed,Max Delivery Distance (miles),Max Deliveries/week,Max Deliveries/day
2,El Merkury,https://goo.gl/maps/tejVM4WbUxguxpfe9,"S, Su",2,3,2
3,On Point,https://goo.gl/maps/CNyUQfXe59PdPmVv6,F,5,4,2
4,Baology,https://g.page/baology?share,None,5,5,1
5,Sidecar,https://goo.gl/maps/cnKkYkWhnVcgRSAK8,None,5,5,1
6,Double Knot,https://goo.gl/maps/ptfobLMFXftCsZ9w5,None,10,5,2
7,Destination Dogs,https://goo.gl/maps/zf6RYMmB9rJnrK2E9,Th,10,4,3
9,Sabrina's Cafe,https://goo.gl/maps/TEY74TYC7Ja1wvX27,W,2,3,1


In [4]:
# Grab the constraints
dist_constraint = rest_df_f['Max Delivery Distance (miles)'].values
slot_constraint = rest_df_f['Max Deliveries/day'].values

In [5]:
# Generate the delivery distance matrix
distance_mat = np.random.rand(len(locs_df), len(rest_df_f))*dist_constraint

In [6]:
# Generate the revenue estimate matrix
revenue_mat = np.random.rand(len(locs_df), len(rest_df_f))*50.0

In [7]:
# Scale the revenue estimate matrix by: dilution factor, preference factor
dilution_mat = np.random.rand(len(locs_df), len(rest_df_f)) # Between 0 and 1 based on # of deliveries to location
preference_mat = np.random.rand(len(locs_df), len(rest_df_f))+1 # Between 1 and 2 
revenue_mat_s = revenue_mat * dilution_mat * preference_mat # Element wise operation

In [8]:
# Create the decision matrix
selections = cp.Variable((revenue_mat_s.shape[1], revenue_mat_s.shape[0]), boolean=True)

In [9]:
# Create the distance constraints
dis_constraint = [distance_mat[i, :]@selections[:, i] <= dist_constraint for i in range(len(locs_df))] # Distnace
# Create the column constraint
col_constraint = [cp.sum(selections, axis=1) <= slot_constraint] # Number of deliveries per day
# Create the row constraint
row_constraint = [cp.sum(selections, axis=0) <= 1] # Each slot has at most 1 scheduled restaurant
# Create the total constraint
tot_constraint = [cp.sum(selections) <= min(revenue_mat_s.shape)]

In [10]:
# Concatenate together
constraints =  dis_constraint + col_constraint + row_constraint + tot_constraint

In [11]:
# Create the objective
objective = cp.Maximize(
    cp.sum(
        [revenue_mat_s[i, :]@selections[:, i] for i in range(len(locs_df))]
    )
)

In [12]:
# Solve it
knapsack_problem = cp.Problem(objective, constraints)
result = knapsack_problem.solve()

In [13]:
# Recover the schedule 
schedule = []
rest_names = rest_df_f['Name'].values
locs_names = locs_df['Name'].values
for idx, x in np.ndenumerate(selections.value):
    if x >= 1.0:
        schedule.append((locs_names[idx[1]], rest_names[idx[0]]))

In [14]:
# Display the results
print("Day of Week: %s, Estimated Revenue: $%0.2f" % (DAY, result))
print("------------------------------------------")
for s in schedule:
    print("Loc: %10s, Rest: %s" % s)

Day of Week: M, Estimated Revenue: $155.52
------------------------------------------
Loc:        HUP, Rest: El Merkury
Loc:  Jefferson, Rest: Sidecar
Loc:       CHOP, Rest: Destination Dogs
Loc:     Presby, Rest: Sabrina's Cafe
